In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import json
import datetime as dt
import matplotlib.dates as mdates
import unidecode

In [2]:
import git
import sys

root_dir = git.Repo('.', search_parent_directories=True).working_tree_dir
sys.path.insert(1, root_dir)

In [3]:
import modcovid as mc
from modcovid import scripts, settings

### Carregando o Arquivo JSON

In [10]:
municipios = [[],[],[]] ## 0: Json File - 1: Nº de municipios - 2: Nome dos municipios
with open('dados/geojs-33-mun.json', encoding='iso-8859-1') as fh:
    municipios[0] = json.load(fh)

FileNotFoundError: [Errno 2] No such file or directory: 'dados/geojs-33-mun.json'

In [11]:
municipios[1] = len(municipios[0]['features']) ## número total de municipios no estado

TypeError: list indices must be integers or slices, not str

### Retirando os nomes dos municipios

In [12]:
for i in range(municipios[1]):
    municipios[2].append(municipios[0]['features'][i]['properties']['name'])

TypeError: 'list' object cannot be interpreted as an integer

### Carregando arquivos de dados do Estado

In [43]:
arquivo_conf_est = f'{root_dir}/dados/RJ/estado_rj/dados_covid/ERJ_Casos.csv'

df_conf_est = pd.read_csv(arquivo_conf_est)

In [14]:
set(df_conf_est['classificacao'].values)

{'CONFIRMADO', 'IGNORADO', 'PROVAVEL', nan}

In [15]:
df_conf_est[df_conf_est['classificacao'] == 'CONFIRMADO']

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta_dt_notif,classificacao,evolucao,dt_obito,comorbidades,dias
0,M,50,RIO DE JANEIRO,RJ,NaN,2020-04-15,CONFIRMADO,NaN,NaN,NaN,18
1,M,76,RIO DE JANEIRO,RJ,NaN,2020-03-24,CONFIRMADO,RECUPERADO,NaN,NaN,40
2,M,68,TERESOPOLIS,RJ,2020-03-24,2020-04-02,CONFIRMADO,OBITO,2020-04-14,SIM,31
3,M,83,RIO DE JANEIRO,RJ,NaN,2020-05-01,CONFIRMADO,NaN,NaN,NaN,2
4,M,41,TERESOPOLIS,RJ,2020-04-12,2020-04-22,CONFIRMADO,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...
13933,M,53,RIO DE JANEIRO,RJ,NaN,2020-04-12,CONFIRMADO,NaN,NaN,NaN,21
13935,F,53,RIO DE JANEIRO,RJ,NaN,2020-04-09,CONFIRMADO,NaN,NaN,NaN,24
13937,F,49,RIO DE JANEIRO,RJ,NaN,2020-04-16,CONFIRMADO,NaN,NaN,NaN,17
13938,F,65,RIO DE JANEIRO,RJ,2020-03-20,2020-04-11,CONFIRMADO,NaN,NaN,NaN,22


In [16]:
import yaml

with open(settings.CONFIG_FILE, encoding = 'utf-8') as f:
    configs = yaml.load(f, Loader = yaml.FullLoader)

In [22]:
def set_df(fonte, *args):
    """Carrega o CSV com dados do covid para determinada cidade ou estado e retorna um pandas.DataFrame com os dados

    Parameters
    ----------
    fonte : str ('prefeitura_rj', 'estado_rj')
        Indica qual a fonte dos dados, com isso a função determina qual CSV carregar e como tratar o DataFrame.
        O arquivo a ser carregado é dado em configs.yml
    *args : dict, optional
        Um dicionário com argumentos extras:
            df_break: Se setado para True, muda o retorno para vários DataFrames
    Returns
    -------
    ret_v: Se nenhum argumento opcional for passado, ret_v é uma lista contendo o DataFrame tratado com todos os casos, e a data de atualização dos dados
            df_break == True: ret_v é uma lista com [DataFrame Tratado, [DataFrame Ativos, DataFrame Recuperados, DataFrame Obitos], Data de Atualização]        
    """

    if fonte == 'prefeitura_rj':
        df = pd.read_csv(root_dir + '/' + configs['csv']['rj']['file_loc']['prefeitura'], encoding = 'iso-8859-1', delimiter = ';')
        df.rename(columns = configs['df']['rename']['rj']['prefeitura'], inplace = True)
        dt_att = df['Data_atualização'].values[0]
        df.drop('Data_atualização', axis = 1, inplace = True)
        if args and args[0]['df_break'] == True:
            df_break = []
            for s in configs['df']['status']['rj']['prefeitura']:
                df_break.append(df[df['Evolucao'] == s])
            ret_lst = [df, df_break, dt_att]
        else:
            ret_lst = [df, dt_att]
    if fonte == 'estado_rj':
        df = pd.read_csv(root_dir + '/' + configs['csv']['rj']['file_loc']['estado'])
        df = df[df['classificacao'] == 'CONFIRMADO']
        df.rename(columns = configs['df']['rename']['rj']['estado'], inplace = True)
        for drop in configs['df']['droppable']['rj']['estado']:
            df.drop(drop, axis = 1, inplace = True)
        if args and args[0]['df_break'] == True:
            df_break = []
            for s in configs['df']['status']['rj']['estado']:
                df_break.append(df[df['Evolucao'] == s])
            ret_lst = [df, df_break]
        else:
            ret_lst = df
    return ret_lst

In [36]:
configs['df']['status']['rj']['estado']

['Internado', 'Recuperado', 'Óbito']

In [27]:
df2, df_break = set_df('estado_rj', {'df_break': True})

In [28]:
lst = []

In [29]:
for i in lst:
    print('')

In [30]:
df2

,Sexo,FaixaEtaria,Municipio,DataSintomas,Data,Evolucao,DataObito,Comorbidades,Dias
0,M,50,RIO DE JANEIRO,NaN,2020-04-15,NaN,NaN,NaN,18
1,M,76,RIO DE JANEIRO,NaN,2020-03-24,RECUPERADO,NaN,NaN,40
2,M,68,TERESOPOLIS,2020-03-24,2020-04-02,OBITO,2020-04-14,SIM,31
3,M,83,RIO DE JANEIRO,NaN,2020-05-01,NaN,NaN,NaN,2
4,M,41,TERESOPOLIS,2020-04-12,2020-04-22,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...
13933,M,53,RIO DE JANEIRO,NaN,2020-04-12,NaN,NaN,NaN,21
13935,F,53,RIO DE JANEIRO,NaN,2020-04-09,NaN,NaN,NaN,24
13937,F,49,RIO DE JANEIRO,NaN,2020-04-16,NaN,NaN,NaN,17
13938,F,65,RIO DE JANEIRO,2020-03-20,2020-04-11,NaN,NaN,NaN,22


In [38]:
df_conf_est

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta_dt_notif,classificacao,evolucao,dt_obito,comorbidades,dias
0,M,50,RIO DE JANEIRO,RJ,NaN,2020-04-15,CONFIRMADO,NaN,NaN,NaN,18
1,M,76,RIO DE JANEIRO,RJ,NaN,2020-03-24,CONFIRMADO,RECUPERADO,NaN,NaN,40
2,M,68,TERESOPOLIS,RJ,2020-03-24,2020-04-02,CONFIRMADO,OBITO,2020-04-14,SIM,31
3,M,83,RIO DE JANEIRO,RJ,NaN,2020-05-01,CONFIRMADO,NaN,NaN,NaN,2
4,M,41,TERESOPOLIS,RJ,2020-04-12,2020-04-22,CONFIRMADO,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...
13935,F,53,RIO DE JANEIRO,RJ,NaN,2020-04-09,CONFIRMADO,NaN,NaN,NaN,24
13936,F,40,PIRAI,RJ,2020-03-24,2020-03-25,IGNORADO,NaN,NaN,NaN,39
13937,F,49,RIO DE JANEIRO,RJ,NaN,2020-04-16,CONFIRMADO,NaN,NaN,NaN,17
13938,F,65,RIO DE JANEIRO,RJ,2020-03-20,2020-04-11,CONFIRMADO,NaN,NaN,NaN,22


In [54]:
di = {'RECUPERADO': 'Recuperado', 'recuperado': 'Recuperado', 'INTERNADO': 'Internado', 'internado': 'Internado', 'OBITO': 'Óbito', 'obito': 'Óbito'}

In [56]:
df_conf_est['evolucao'].map(di).fillna(df_conf_est['evolucao']))

{'Internado', 'NAO', 'Recuperado', nan, 'Óbito'}

In [64]:
df_conf_est['evolucao'].replace(pd.Series(di), inplace = True)

In [65]:
df_conf_est

,sexo,idade,municipio_res,uf,dt_sintoma,dt_coleta_dt_notif,classificacao,evolucao,dt_obito,comorbidades,dias
0,M,50,RIO DE JANEIRO,RJ,NaN,2020-04-15,CONFIRMADO,NaN,NaN,NaN,18
1,M,76,RIO DE JANEIRO,RJ,NaN,2020-03-24,CONFIRMADO,Recuperado,NaN,NaN,40
2,M,68,TERESOPOLIS,RJ,2020-03-24,2020-04-02,CONFIRMADO,Óbito,2020-04-14,SIM,31
3,M,83,RIO DE JANEIRO,RJ,NaN,2020-05-01,CONFIRMADO,NaN,NaN,NaN,2
4,M,41,TERESOPOLIS,RJ,2020-04-12,2020-04-22,CONFIRMADO,NaN,NaN,NaN,11
...,...,...,...,...,...,...,...,...,...,...,...
13935,F,53,RIO DE JANEIRO,RJ,NaN,2020-04-09,CONFIRMADO,NaN,NaN,NaN,24
13936,F,40,PIRAI,RJ,2020-03-24,2020-03-25,IGNORADO,NaN,NaN,NaN,39
13937,F,49,RIO DE JANEIRO,RJ,NaN,2020-04-16,CONFIRMADO,NaN,NaN,NaN,17
13938,F,65,RIO DE JANEIRO,RJ,2020-03-20,2020-04-11,CONFIRMADO,NaN,NaN,NaN,22


In [35]:
df_break[3]

IndexError: list index out of range

In [8]:
df = read_data(df_conf_est, fonte = 'estado')

NameError: name 'read_data' is not defined

obs: na função read_data, inutilizamos todos os dados com 'NaN' na coluna de datas (isto é, casos sem data).

In [ ]:
df[df['Data'] == '27/02/2020'] ## inicio: primeiro dia

In [ ]:
dic_municipios = {v:unidecode.unidecode(v.upper()) for v in municipios[2]}

In [ ]:
lst_conf = []
for i in range(municipios[1]):
    lst_conf.append(get_data(local = dic_municipios[municipios[2][i]], df = df, fonte = 'estado', 
                             T_fim = '23-04-2020', to_print = False))

In [ ]:
## Preparando o Dicionario
lst_dados = []
for i in range(municipios[1]):
    lst_dados.append(lst_conf[i][0][-2])

In [ ]:
start = dt.datetime.strptime('29-03-2020', "%d-%m-%Y")
then = dt.datetime.strptime('23-04-2020', "%d-%m-%Y")
days = mdates.drange(start,then,dt.timedelta(days=1))
dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]

### Montando o Dicionario e o DataFrame

In [ ]:
dic = {'Cidade': municipios[2], 'Casos Confirmados': lst_dados}

In [ ]:
df_confirmados = pd.DataFrame(dic)

## Separando o DataFrame de semanas

In [ ]:
## fonte: https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Rio_de_Janeiro_por_popula%C3%A7%C3%A3o
pop_municipios = pd.read_csv('dados/pop_municipiosRJ.csv')

In [ ]:
dic_populacao = {m:int(d.replace('.','').replace(' ', '')) for m,d in zip(pop_municipios['Municipio'].values,pop_municipios['Dados'].values)}

In [ ]:
def get_dias(T_start, T_fim):
    start = dt.datetime.strptime(T_start, "%d-%m-%Y")
    then = dt.datetime.strptime(T_fim, "%d-%m-%Y")
    days = mdates.drange(start,then,dt.timedelta(days=1))
    dias = [str(mdates.num2date(v)).split(' ')[0] for v in days]
    dias = [f'{d[-2:]}/{d[5:7]}/{d[:4]}' for d in dias]
    return dias

In [ ]:
T_start = '26-02-2020'
T_fim = '22-04-2020'

In [ ]:
df = read_data(df_conf_est, fonte = 'estado')
dic_municipios = {v:unidecode.unidecode(v.upper()) for v in municipios[2]}
lst_conf = []
for i in range(municipios[1]):
    lst_conf.append(get_data(local = dic_municipios[municipios[2][i]], df = df, fonte = 'estado', T_start = T_start, T_fim = T_fim, to_print = False))

In [ ]:
lst_conf_acumul_semanas = []
lst_conf_novos_semanas = [] 
for i in range(8):
    if i == 7:
        lst_conf_acumul_semanas.append([v[0][-1] * 1e5/dic_populacao[municipios[2][j]] for j,v in enumerate(lst_conf)])
        lst_conf_novos_semanas.append([sum(v[1][i*7:]) * 1e5/dic_populacao[municipios[2][j]] for j,v in enumerate(lst_conf)])
    else:
        lst_conf_acumul_semanas.append([v[0][(i+1)*(7)] * 1e5/dic_populacao[municipios[2][j]] for j,v in enumerate(lst_conf)])
        lst_conf_novos_semanas.append([sum(v[1][i*7:(i+1)*7]) * 1e5/dic_populacao[municipios[2][j]] for j,v in enumerate(lst_conf)])

In [ ]:
lst_conf_novos_semanas = np.array(lst_conf_novos_semanas).flatten()
lst_conf_acumul_semanas = np.array(lst_conf_acumul_semanas).flatten()

In [ ]:
lst_semanas = [1]*92 + [2]*92 + [3]*92 + [4]*92 + [5]*92 + [6]*92 + [7]*92 + [8]*92

In [ ]:
dias = get_dias('27-02-2020', '23-04-2020')[::7]
dias_fim = get_dias('26-02-2020', '23-04-2020')[::7][1:]
lst_dias_semanas = [dias[0]]*92 + [dias[1]]*92 + [dias[2]]*92 + [dias[3]]*92 + [dias[4]]*92 + [dias[5]]*92 + [dias[6]]*92 + [dias[7]]*92
#n_semana = 2

In [ ]:
dic_semanas = {'Semana': lst_semanas, 'Municipio': municipios[2]*8, 'Casos Novos': lst_conf_novos_semanas, 'Casos Acumulados': lst_conf_acumul_semanas,
               'Início da semana': lst_dias_semanas}


In [ ]:
df_semanas = pd.DataFrame(dic_semanas)

In [ ]:
#df_semanas.to_csv('dados/geo_plot_erj_semanas_pronto.csv')

## Fazendo o html de casos confirmados acumulados e casos confirmados novos:

In [ ]:
df_semanas = df_semanas.round(decimals = 3) # pra usar só 3 casas decimais nos casos relativos

In [ ]:
lst_conf_acumul_n = []
lst_conf_novos_n = [] 
for i in range(8):
    if i == 7:
        lst_conf_acumul_n.append([v[0][-1] for j,v in enumerate(lst_conf)])
        lst_conf_novos_n.append([sum(v[1][i*7:]) for j,v in enumerate(lst_conf)])
    else:
        lst_conf_acumul_n.append([v[0][(i+1)*(7)] for j,v in enumerate(lst_conf)])
        lst_conf_novos_n.append([sum(v[1][i*7:(i+1)*7]) for j,v in enumerate(lst_conf)])

In [ ]:
lst_conf_novos_n = np.array(lst_conf_novos_n).flatten()
lst_conf_acumul_n = np.array(lst_conf_acumul_n).flatten()

In [ ]:
dic_n =  {'Semana': lst_semanas, 'Municipio': municipios[2]*8, 'Casos Novos': lst_conf_novos_n, 'Casos Acumulados': lst_conf_acumul_n,
               'Início da semana': lst_dias_semanas}

In [ ]:
df_n = pd.DataFrame(dic_n)

In [ ]:
import plotly.graph_objects as go

In [ ]:
data_novos = []
data_acumul = []
for i in range(1,9):
    n_semana = i
    
    data_1 = [go.Choropleth(z = df_semanas.loc[df_semanas['Semana'] == n_semana]['Casos Novos'], 
                          geojson = municipios[0],
                          locations= df_semanas.loc[df_semanas['Semana'] == n_semana]['Municipio'],
                           featureidkey = 'properties.name',
                          colorscale = "ylorrd",
                           colorbar_title = 'rel. 100 mil hab.',
                            visible = (i==1),
                            name = '',
                            hovertemplate = df_semanas.loc[df_semanas['Semana'] == n_semana]['Municipio'].astype(str) + '<br>' + \
                            'Casos rel.: ' + df_semanas.loc[df_semanas['Semana'] == n_semana]['Casos Novos'].astype(str) + '<br>' + \
                               'Casos: ' + df_n.loc[df_n['Semana'] == n_semana]['Casos Novos'].astype(str),
                          zmin = 0,
                          zmax = 10)]
    
    data_2 = [go.Choropleth(z = df_semanas.loc[df_semanas['Semana'] == n_semana]['Casos Acumulados'], 
                          geojson = municipios[0],
                          locations= df_semanas.loc[df_semanas['Semana'] == n_semana]['Municipio'],
                           featureidkey = 'properties.name',
                          colorscale = "ylorrd",
                           colorbar_title = 'rel. 100 mil hab.',
                            visible = False,
                            name = '',
                             hovertemplate = df_semanas.loc[df_semanas['Semana'] == n_semana]['Municipio'].astype(str) + '<br>' + \
                            'Casos rel.: ' + df_semanas.loc[df_semanas['Semana'] == n_semana]['Casos Acumulados'].astype(str) + '<br>' + \
                               'Casos: ' + df_n.loc[df_n['Semana'] == n_semana]['Casos Acumulados'].astype(str),
                            zmin = 0,
                            zmax = 31)]
    
    
    
    data_novos+= data_1
    data_acumul+= data_2
    
data = data_novos + data_acumul

In [ ]:
titles = [f'Casos <b>novos</b> de {dias[i -1]} até {dias_fim[i-1]} <br> no estado do Rio de Janeiro' for i in range(1,9)] + [f'Casos <b>acumulados</b> até {dias_fim[i-1]} <br> no estado do Rio de Janeiro' for i in range(1,9)]
labels = [f'{dias[i -1]} até {dias_fim[i-1]}' for i in range(1,9)] + [f'até {dias_fim[i-1]}' for i in range(1,9)]

In [ ]:
updatemenus = list([dict(active = 0,
                         buttons = list([dict(label = labels[i],
                                              method = 'update', 
                                              args = [{'visible':  [ j == i for j in range(len(data))] },
                                                      {'title': titles[i]}
                                                      ]) for i in range(len(data))])) ])

In [ ]:
%%time
fig = go.Figure(data= data)
fig.update_layout(updatemenus = updatemenus, title = dict(text = 'Casos <b>novos</b> de 27/02/2020 até 04/03/2020 <br> no estado do Rio de Janeiro', x = 0.5))
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
#with open("Estado_plots.html", "w") as f:
#    f.write(fig.to_html())

## continuação do notebook: 
## fazendo subplots com os casos confirmados acumulados

In [ ]:
for i in range(1,9):
    n_semana = i
    fig = px.choropleth(df_semanas.loc[df_semanas['Semana'] == n_semana], geojson = municipios[0], locations='Municipio', color='Casos Acumulados',
                               color_continuous_scale = "ylorrd",
                               range_color=(0,60),featureidkey="properties.name",
                               labels={'Casos Acumulados':'Casos por 100k habitantes'}
                              )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title_text = f'Casos Confirmados por 100 mil habitantes <br>semana do dia {dias[n_semana -1]} até o dia {dias_fim[n_semana-1]}',
                      title_y = 0.93, title_x = 0.4)
    fig.show()

## Animação dos casos confirmados acumulados

In [ ]:
fig = px.choropleth(df_semanas, geojson = municipios[0], locations='Municipio', color='Casos Acumulados',
                           color_continuous_scale = "ylorrd", animation_frame = 'Início da semana', animation_group = 'Municipio',
                           range_color=(0,60),featureidkey="properties.name",
                           labels={'Casos Acumulados':'Casos por 100k habitantes'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, title_text = f'Casos Confirmados por 100 mil habitantes', 
                  title_y = 0.93, title_x = 0.1)
fig.show()